# Projekt: Rozpoznawanie i Odporność Modelu na Znaki Drogowe

### Struktura projektu


1. **Modularność**: Kod podzielony na sekcje dla przetwarzania danych, trenowania modelu, testowania i generowania ataków adversarialnych.
2. **Reużywalność**: Możliwość łatwego dostosowania do innych zbiorów danych.


## Importowanie bibliotek

In [2]:

import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from foolbox import PyTorchModel, accuracy, samples, attacks
import torch
import torchvision.transforms as transforms


ModuleNotFoundError: No module named 'sklearn'

## 1. Przetwarzanie danych

In [ ]:

# Ścieżka do zbioru danych
data_dir = 'GTSRB/Final_Training/Images'
num_classes = 43  # Liczba klas w zestawie danych GTSRB

# Wczytywanie danych
def load_data(data_dir, img_size=(32, 32)):
    images = []
    labels = []
    for class_id in range(num_classes):
        class_dir = os.path.join(data_dir, '{0:05d}'.format(class_id))
        if not os.path.exists(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(class_id)
    return np.array(images), np.array(labels)

images, labels = load_data(data_dir)
# Normalizacja danych
images = images.reshape(-1, 32, 32, 1) / 255.0
labels = to_categorical(labels, num_classes)
# Podział na zestawy treningowe i walidacyjne
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)


## 2. Definicja modelu

In [ ]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## 3. Trenowanie modelu

In [ ]:

# Augmentacja danych
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

# Trenowanie modelu
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    epochs=15,
                    validation_data=(X_val, y_val))


## 4. Testowanie modelu

In [ ]:

# Ocena modelu
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f'Loss: {val_loss}, Accuracy: {val_acc}')

# Macierz pomyłek
y_pred = model.predict(X_val)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_val, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)


## 5. Generowanie ataków adversarialnych

In [ ]:

# Konwersja modelu do formatu Foolbox
fmodel = PyTorchModel(model, bounds=(0, 1))

# Pobieranie przykładowych danych
images_tensor = torch.tensor(X_val, dtype=torch.float32).permute(0, 3, 1, 2)
labels_tensor = torch.tensor(np.argmax(y_val, axis=1))

# Generowanie ataku FGSM
attack = attacks.FGSM()
adversarial_examples = attack(fmodel, images_tensor, labels_tensor)


## 6. Ewaluacja odporności

In [ ]:

# Ocena modelu na danych adversarialnych
adv_predictions = fmodel(adversarial_examples)
adv_accuracy = accuracy(fmodel, adversarial_examples, labels_tensor)
print(f'Adversarial Accuracy: {adv_accuracy}')
